Todo:
- scrapovat úmrtí (349_3.html)

In [227]:
import os
import re
import json
from bs4 import BeautifulSoup

In [228]:
odkud = "downloads/poslanectvo"

In [229]:
kam = "data_raw/poslanectvo"
os.makedirs(kam, exist_ok=True)

In [230]:
stazene = os.listdir(odkud)

In [231]:
def najdi_datum(retezec):
    retezec_cisty = retezec.replace('\xa0', ' ').replace('\u00a0', ' ')
    if 'v roce' not in retezec_cisty:
        datum = re.search(r'\d{1,2}\.\s\d{1,2}\.\s\d{4}', retezec_cisty).group().replace('\xa0',' ')
    else:
        datum = re.search(r'\d{4}', retezec_cisty).group()
    return datum

In [232]:
najdi_datum('Narozena: v roce 1954')

'1954'

In [233]:
def rozlouskni_odrazku(odrazka):
    odrazka_normalizovana = odrazka.text.replace('\xa0', ' ').replace('\u00a0', ' ')
    polozka = {}
    pozice = [' člen',' místopředsed',' předsed']
    for p in pozice:
        if p in odrazka_normalizovana:
            polozka['funkce'] = p.strip()
    if odrazka.find('a'):
        dodatek = ''
        if 'Skupina ČR - ' in odrazka_normalizovana:
            dodatek = 'Skupina ČR - '
        polozka['co'] = f"{dodatek}{odrazka.find('a').text}"
        if ' od ' in odrazka_normalizovana:
            polozka['od'] = najdi_datum(odrazka_normalizovana.split(' od ')[1])
        if ' do ' in odrazka_normalizovana:
            polozka['do'] = najdi_datum(odrazka_normalizovana.split(' do ')[1])
        return polozka

In [234]:
def oscrapuj_poslance(soubor):
    
    with open(os.path.join(odkud, soubor), 'r', encoding='utf-8') as poslanec:
        poslanec = poslanec.read()
        soup = BeautifulSoup(poslanec)
        if 'Volební kraj' in soup.text:
            tabulka = {}
            tabulka['id'] = soubor.split('_')[0]
            tabulka['obdobi'] = soubor.split("_")[1].split('.html')[0]
            tabulka['soubor'] = soubor
            tabulka['jmeno'] = soup.find('title').text
            tabulka['kraj'] = soup.text.split('Volební kraj: ')[1].split(' ')[0]
            if 'poslanec ' in soup.text:
                tabulka['gender'] = 'm'
                tabulka['obdobi_od'] = najdi_datum(soup.text.split('poslanec ')[1][0:100])
                try:
                    tabulka['obdobi_do'] = najdi_datum(soup.text.split('poslanec ')[1].split("člen")[0][10:100])
                except:
                    tabulka['obdobi_do'] = None
            if 'poslankyně ' in soup.text:
                tabulka['gender'] = 'f'
                tabulka['obdobi_od'] = najdi_datum(soup.text.split('poslankyně ')[1][0:100])
                try:
                    tabulka['obdobi_do'] = najdi_datum(soup.text.split('poslankyně ')[1].split("Klub")[0][10:100])
                except:
                    tabulka['obdobi_do'] = None
            kandidatka = None
            if 'na kandidátce:' in soup.text:
                kandidatka = soup.text.split('na kandidátce:')[1].split('Seznam')[0].strip()
            navrhovatel = None
            if ', navrhovatel' in str(kandidatka):
                kandidatka, navrhovatel = kandidatka.split(', navrhovatel')[0].strip(), kandidatka.split(', navrhovatel')[1].strip()
            tabulka['kandidatka'], tabulka['navrhovatel'] = kandidatka, navrhovatel
    
            popisek = soup.find(class_='figcaption').text
            if 'naroze' in popisek[0:50].lower():
                tabulka['narozeni'] = najdi_datum(popisek[0:50])
                
            odrazky = soup.find_all('li')
            clenstvi = []
            for o in odrazky:
                try:
                    zpracovana_odrazka = rozlouskni_odrazku(o)
                except:
                    zpracovana_odrazka = None
                        # print(f"problém s odrážkou v souboru {soubor}")
                if zpracovana_odrazka != None:
                    if zpracovana_odrazka.get('od'):
                        clenstvi.append(rozlouskni_odrazku(o))
            tabulka['clenstvi'] = clenstvi
        
            return tabulka

In [235]:
oscrapuj_poslance('4_9.html')

{'id': '4',
 'obdobi': '9',
 'soubor': '4_9.html',
 'jmeno': 'Marek Benda',
 'kraj': 'Hlavní',
 'gender': 'm',
 'obdobi_od': '9. 10. 2021',
 'obdobi_do': None,
 'kandidatka': 'SPOLU',
 'navrhovatel': 'ODS',
 'narozeni': '10. 11. 1968',
 'clenstvi': [{'funkce': 'člen',
   'co': 'Mandátový a imunitní výbor',
   'od': '10. 11. 2021'},
  {'funkce': 'člen', 'co': 'Zahraniční výbor', 'od': '23. 11. 2021'},
  {'funkce': 'člen', 'co': 'Ústavně-právní výbor', 'od': '23. 11. 2021'},
  {'funkce': 'člen', 'co': 'Organizační výbor', 'od': '23. 11. 2021'},
  {'funkce': 'místopředsed',
   'co': 'Ústavně-právní výbor',
   'od': '24. 11. 2021'},
  {'funkce': 'člen',
   'co': 'Podvýbor pro náhradní výchovnou péči',
   'od': '15. 12. 2021'},
  {'funkce': 'člen',
   'co': 'Podvýbor pro myslivost, rybářství, včelařství, zahrádkářství a chovatelství',
   'od': '18. 1. 2022'},
  {'funkce': 'člen', 'co': 'Podvýbor pro justici', 'od': '19. 1. 2022'},
  {'co': 'Podvýbor pro myslivost, rybářství, včelařství, zah

In [236]:
oscrapuj_poslance('100_1.html')

{'id': '100',
 'obdobi': '1',
 'soubor': '100_1.html',
 'jmeno': 'JUDr. Hana Kordová Marvanová',
 'kraj': 'Hlavní',
 'gender': 'f',
 'obdobi_od': '6. 6. 1992',
 'obdobi_do': '6. 6. 1996',
 'kandidatka': None,
 'navrhovatel': None,
 'narozeni': '1962',
 'clenstvi': [{'funkce': 'člen',
   'co': 'Ústavně právní výbor',
   'od': '6. 6. 1992',
   'do': '6. 6. 1996'},
  {'funkce': 'místopředsed',
   'co': 'Ústavně právní výbor',
   'od': '6. 6. 1992',
   'do': '6. 6. 1996'},
  {'funkce': 'člen',
   'co': 'Stálá komise pro sdělovací prostředky',
   'od': '22. 4. 1995',
   'do': '6. 6. 1996'},
  {'funkce': 'člen',
   'co': 'Poslanecký klub Občanské demokratické strany',
   'od': '6. 6. 1992',
   'do': '6. 6. 1996'}]}

In [250]:
stazene = os.listdir(odkud)

for s in stazene:
    try:
        poslanec = oscrapuj_poslance(s)
    except Exception as e:
        print(s)
        print(e)
        break
    if poslanec != None:
        with open(os.path.join(kam, s.replace('.html','.ndjson')), "w+", encoding="utf-8") as export:
            export.write(json.dumps(poslanec))

print("hotovo dvacet")

hotovo dvacet
